In [1]:
import os 
import streamlit
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
import dotenv 
dotenv.load_dotenv()


USER_AGENT environment variable not set, consider setting it to identify your requests.


True

# login hugging face and setup llm

In [2]:

from huggingface_hub import login
llm=ChatGroq(
    groq_api_key=os.getenv('GROQ_API_KEY'),
    model_name='llama-3.1-70b-versatile',
    temperature=0.2,
    
    
)
login(token=os.getenv('HUGGINGFACEHUB_API_TOKEN'),add_to_git_credential=True)


d:\anaconda\envs\genai\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token is valid (permission: fineGrained).
Your token has been saved in your configured git credential helpers (manager).
Your token has been saved to C:\Users\26amr\.cache\huggingface\token
Login successful


# split the data into chunks 

In [3]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, load_index_from_storage, StorageContext
from llama_index.vector_stores.faiss import FaissVectorStore
# import faiss
# embeddings=OllamaEmbeddings()
# loader=WebBaseLoader('https://en.wikipedia.org/wiki/Elon_Musk')
# docs=loader.load()
# text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
# documents=text_splitter.split_documents(docs)
# print(documents)

documents=SimpleDirectoryReader('../data/').load_data()



In [4]:
import faiss
d=1536
faiss_index=faiss.IndexFlatL2(d)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader 
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core import Settings
embed_model=HuggingFaceEmbedding(
    model_name='BAAI/bge-small-en-v1.5'
)
# document_text=[doc.page_content for doc in documents]
# embeddings=[embed_model.embed_query(text) for text in document_text]



In [5]:
from llama_index.core import Settings

Settings.chunk_size=1024
Settings.embed_model=embed_model
Settings.llm=llm

In [6]:
index=VectorStoreIndex.from_documents(documents=documents,embed_model=Settings.embed_model)
# save the data 
index.storage_context.persist('../data/')


In [7]:
# load index from storage
storage_context=StorageContext.from_defaults(persist_dir='../data/')
index=load_index_from_storage(storage_context)

# create a retriever or query engine 
retriever=index.as_retriever()
query_engine=index.as_query_engine()


In [8]:
response=query_engine.query('what are the rules i should follow?')
response.response

d:\anaconda\envs\genai\lib\site-packages\llama_index\llms\langchain\base.py:95: LangChainDeprecationWarning: The method `BaseChatModel.predict_messages` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  lc_message = self._llm.predict_messages(messages=lc_messages, **kwargs)


"To determine domicile, the rules are as follows: \n\n1. A person's residence is the place of their domicile, or the place where their residence is fixed without any present intention of removing therefrom.\n2. A person may acquire a domicile if they are personally present in a place and elect that as their home, even if they never designed to remain there always, but designed at the end of some short time to remove and acquire another.\n3. It is sufficient if they intend to make it their home in good faith for the time being, to the exclusion of all others.\n4. It is not necessary that there should be an intention to remain permanently at the chosen domicile."

# bot context 

In [9]:
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core.prompts import ChatPromptTemplate
from llama_index.core.memory import BaseMemory
# memory=BaseMemory()
language= 'French'
# Text QA Prompt
chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            """ 
            You are an experienced immigration lawyer providing detailed legal advice in {language}. 
            Carefully review the following documents to extract relevant information and address the client's question 
            Analyze any legal regulations, risks, or implications that may apply, and provide guidance on the next steps.

                Instructions:
                1. Identify key legal issues or relevant regulations from the provided documents.
                2. Explain how the information applies to the client’s situation.
                3. Highlight any significant risks, legal obligations, or considerations the client should be aware of.
                4. Offer clear advice on the next steps, considering short-term and long-term outcomes.
                5. Note any additional documentation, evidence, or forms the client should prepare

                Please provide a structured and thorough response, clearly addressing each point in a way that helps the client understand their legal standing and options in {language}.
            """
        ),
    ),
    ChatMessage(
        role=MessageRole.USER,
        content=(
            "Context information is below.\n"
            "---------------------\n"
            "{context_str}\n"
            "---------------------\n"
            "Given the context information and not prior knowledge, "
            "answer the query.\n"
            "Query: {query_str}\n"
            "Answer: "
        ),
    ),
]

text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

# Refine Prompt
chat_refine_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            "You are an expert Q&A system that strictly operates in two modes "
            "when refining existing answers:\n"
            "1. **Rewrite** an original answer using the new context.\n"
            "2. **Repeat** the original answer if the new context isn't useful.\n"
            "Never reference the original answer or context directly in your answer.\n"
            "When in doubt, just repeat the original answer."
        ),
    ),
    ChatMessage(
        role=MessageRole.USER,
        content=(
            "New Context: {context_msg}\n"
            "Query: {query_str}\n"
            "Original Answer: {existing_answer}\n"
            "New Answer: "
        ),
    ),
]
refine_template = ChatPromptTemplate(chat_refine_msgs)


In [10]:
chat_text_qa_msgs[0]

ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content=" \n            You are an experienced immigration lawyer providing detailed legal advice in {language}. \n            Carefully review the following documents to extract relevant information and address the client's question \n            Analyze any legal regulations, risks, or implications that may apply, and provide guidance on the next steps.\n\n                Instructions:\n                1. Identify key legal issues or relevant regulations from the provided documents.\n                2. Explain how the information applies to the client’s situation.\n                3. Highlight any significant risks, legal obligations, or considerations the client should be aware of.\n                4. Offer clear advice on the next steps, considering short-term and long-term outcomes.\n                5. Note any additional documentation, evidence, or forms the client should prepare\n\n                Please provide a structured and t

In [11]:
print(
    index.as_query_engine(
        text_qa_template=text_qa_template, refine_template=refine_template
    ).query("what are some things i sohuld avoid?")
)

Based on the provided context information, here are some things you should avoid:

1. **Registering invalid bonds**: As the Comptroller, you have ministerial and executive functions, and registering invalid bonds knowing them to be invalid would be a violation of the law and a disregard of your duty (Page 50, REPORT OF ATTORNEY GENERAL).

2. **Paying officers' costs without sufficient funds**: While it is competent for the County Judge to draw a warrant in favor of officers for costs due to them when sufficient funds are available, you should avoid doing so without ensuring that the amount of costs has been paid (ATTOHNEY GENERAL's OFFICE, AUSTIN, April 14, 1893).

3. **Ignoring the law governing expenditures**: When auditing and paying claims, ensure that there is a law authorizing the expenditure and an appropriation by the Legislature to cover the costs (ATTORNEY GENERAL'S OFFICE, AUSTIN, June 22, 1893).

4. **Disregarding the authority of the Printing Board**: The Printing Board ha

In [12]:
# include chat history and memory
from llama_index.core.memory import ChatMemoryBuffer

memory= ChatMemoryBuffer.from_defaults(token_limit=1000)
chat_engine=index.as_chat_engine(
    chat_mode='context',
    memory=memory,
    system_prompt=(text_qa_template,refine_template),
)


In [13]:
response=chat_engine.chat('what was the latest law passed?')
response.response

'The latest law passed was "An act to amend article 2578 and article 2581 of chapter 10 of the Revised Statutes of the State of Texas, and to add thereto article 2589a, providing for a hypothecation of lands belonging to an estate in the hands of a guardian, and article 2589b providing for the novation of existing indebtedness of estates in guardianship."'

In [14]:
response=chat_engine.chat('when was it passed?')
response.response

'The act was approved May 2, 1893.'

In [15]:
response=chat_engine.chat('can you tell me more')
response.response

'The act "An act to amend article 2578 and article 2581 of chapter 10 of the Revised Statutes of the State of Texas, and to add thereto article 2589a, providing for a hypothecation of lands belonging to an estate in the hands of a guardian, and article 2589b providing for the novation of existing indebtedness of estates in guardianship" was passed in response to a message from the Governor, which stated that under existing law, real property under administration or guardianship could not be sold on longer time than twelve months, and that it would be well to permit real estate to be sold on a longer time with good security under suitable restrictions.\n\nThe act was part of the General Laws of the Special Session of the Twenty-second Legislature. However, the question of its constitutionality was raised, as the subject of the act was not mentioned in the Governor\'s proclamation calling the special session.'

In [16]:
response=chat_engine.chat('what are some things i should avoid in texas?')
response.response

"Based on the context information, it appears that some things you should avoid in Texas include:\n\n1. Becoming a surety on the bond of any county officer, as it is against public policy for a county commissioner to become a surety on the bond of any county officer.\n2. Contracting a debt against the county without the consent and authority of the Commissioners Court, as the city authorities had no power to contract a debt against the county without the consent and authority of the Commissioners Court.\n3. Issuing county bonds that are not sold, but are paid directly to the contractor, as the county treasurer is not entitled to commissions on bonds paid directly to the contractor.\n\nIt's also worth noting that the context information is from the late 19th century, so some of these things may not be relevant today."

In [17]:
response=chat_engine.chat('what are some things i should avoid?')
print(response.response)

Based on the context information, it appears that some things you should avoid include:

1. Registering bonds that are invalid, as the Comptroller has ministerial and executive functions and should not register bonds that are invalid.
2. Paying officers' costs without proper authorization, as the County Judge should only pay officers' costs after the amount of costs has been paid and the fine has been paid.
3. Approving claims for advertising in newspapers without proper authorization, as the Printing Board has the authority and duty to examine and audit claims against the State for printing department reports.
4. Filling vacancies in the office of District Clerk or County Clerk without proper authorization, as the authority to fill vacancies is vested in the District Judge or the Commissioners' Court, depending on the population of the county.

It's also worth noting that these are specific to the context of Texas law in the late 19th century, and may not be relevant today.


In [18]:
response=chat_engine.chat('could you elaborate on the 7th point')
print(response.response)

There is no 7th point mentioned in the context information. The context information appears to be a collection of letters and opinions from the Attorney General's office in Texas, and it does not contain a numbered list of points. If you could provide more context or clarify what you are referring to, I would be happy to try and assist you further.


In [19]:
response=chat_engine.chat('what if i dont follow the protocols?')
print(response.response)

If you don't follow the protocols, it may lead to invalid or unenforceable actions. For example, if a bond is issued without proper authorization or if a canvassing board does not follow the proper procedures, the bond or the election results may be deemed invalid.

Additionally, if you don't follow the protocols, you may be held liable for any damages or consequences that arise from your actions. For instance, if a county judge fails to follow the proper procedures for paying officers' costs, they may be personally liable for any improper payments.

It's also worth noting that failing to follow protocols can undermine the integrity of the electoral process and the administration of justice. It's essential to follow established protocols and procedures to ensure that actions are valid, enforceable, and in the best interest of the public.


# change language

In [20]:
# try